## Table II Accuracy and Sparsity Comparison
### Preparation
Change the `res_path` variable to the path of simulator results. 
Change the `acc_path` variable to the path of algorithm evaluation results

In [3]:
res_path = '../../../simulator/example_sim_results/'
acc_path = '../../../algorithm/example_output/'

Accuracy and Computation Sparsity of Focus and Baselines.
Computation sparsity is calculated using number of operations: $1 - \frac{\text{real num OPS}}{\text{dense systolic array num OPS}} \times 100\%$

In [4]:
import pandas as pd
import numpy as np

model_display = {'llava_vid': 'Llava-Vid', 'llava_onevision': 'Llava-OV', 'minicpm_v': 'MiniCPM'}
dataset_display = {'videomme': 'VMME', 'mlvu': 'MLVU', 'mvbench': 'MVB'}

df_adaptiv = pd.read_csv(res_path + '/main_adaptiv.csv')
df_cmc = pd.read_csv(res_path + '/main_cmc.csv')
df_focus = pd.read_csv(res_path + '/main_focus.csv')
df_acc = pd.read_csv(acc_path + '/accuracy.csv')

df_adaptiv = df_adaptiv[df_adaptiv['model'].isin(model_display.keys()) &
                         df_adaptiv['dataset'].isin(dataset_display.keys())]
df_cmc = df_cmc[df_cmc['model'].isin(model_display.keys()) &
                 df_cmc['dataset'].isin(dataset_display.keys())]
df_focus = df_focus[df_focus['model'].isin(model_display.keys()) &
                     df_focus['dataset'].isin(dataset_display.keys())]

result = []

for model_key in model_display.keys():
    for dataset_key in dataset_display.keys():
        adaptiv_match = df_adaptiv[(df_adaptiv['model'] == model_key) & (df_adaptiv['dataset'] == dataset_key)]
        cmc_match = df_cmc[(df_cmc['model'] == model_key) & (df_cmc['dataset'] == dataset_key)]
        focus_match = df_focus[(df_focus['model'] == model_key) & (df_focus['dataset'] == dataset_key)]

        if focus_match.empty:
            continue

        # dense_ops is stored in each file as the baseline
        dense_ops = focus_match.iloc[0]['dense_ops']
        adaptiv_ops = adaptiv_match.iloc[0]['num_ops'] if not adaptiv_match.empty else np.nan
        cmc_ops = cmc_match.iloc[0]['num_ops'] if not cmc_match.empty else np.nan
        focus_ops = focus_match.iloc[0]['num_ops']

        ff_sparsity = 70.0  # FF sparsity is fixed at 70%
        adaptiv_sparsity = (1 - adaptiv_ops / dense_ops) * 100 if not np.isnan(adaptiv_ops) else np.nan
        cmc_sparsity = (1 - cmc_ops / dense_ops) * 100 if not np.isnan(cmc_ops) else np.nan
        focus_sparsity = (1 - focus_ops / dense_ops) * 100

        acc_match = df_acc[(df_acc['Models'] == model_key) &
                            (df_acc['Dataset'] == dataset_key)]

        if not acc_match.empty:
            dense_acc = acc_match.iloc[0]['Dense']
            ff_acc = acc_match.iloc[0]['FF']
            adaptiv_acc = acc_match.iloc[0]['Adaptiv']
            cmc_acc = acc_match.iloc[0]['CMC']
            focus_acc = acc_match.iloc[0]['Focus']
        else:
            print(f"Warning: No accuracy data for {model_key} - {dataset_key}")
            dense_acc = ff_acc = adaptiv_acc = cmc_acc = focus_acc = np.nan

        acc_row = {
            'Models': model_display[model_key],
            'Dataset': dataset_display[dataset_key],
            'Metric': 'Acc.',
            'Ori.': f'{dense_acc:.2f}' if not np.isnan(dense_acc) else 'N/A',
            'FF': f'{ff_acc:.2f}' if not np.isnan(ff_acc) else 'N/A',
            'Ada.': f'{adaptiv_acc:.2f}' if not np.isnan(adaptiv_acc) else 'N/A',
            'CMC': f'{cmc_acc:.2f}' if not np.isnan(cmc_acc) else 'N/A',
            'Ours': f'{focus_acc:.2f}' if not np.isnan(focus_acc) else 'N/A'
        }

        sparsity_row = {
            'Models': '',
            'Dataset': '',
            'Metric': 'Sparsity',
            'Ori.': '-',
            'FF': f'{ff_sparsity:.2f}',
            'Ada.': f'{adaptiv_sparsity:.2f}' if not np.isnan(adaptiv_sparsity) else 'N/A',
            'CMC': f'{cmc_sparsity:.2f}' if not np.isnan(cmc_sparsity) else 'N/A',
            'Ours': f'{focus_sparsity:.2f}'
        }

        result.append(acc_row)
        result.append(sparsity_row)

df_result = pd.DataFrame(result)

print("="*70)
print("TABLE II: ACCURACY AND COMPUTATION SPARSITY")
print("="*70)
print(df_result.to_string(index=False))
print("="*70)

TABLE II: ACCURACY AND COMPUTATION SPARSITY
   Models Dataset   Metric  Ori.    FF  Ada.   CMC  Ours
Llava-Vid    VMME     Acc. 64.15 62.00 62.44 62.52 62.74
                  Sparsity     - 70.00 52.15 58.62 82.82
Llava-Vid    MLVU     Acc. 67.74 65.38 65.94 65.17 65.99
                  Sparsity     - 70.00 32.52 42.46 78.26
Llava-Vid     MVB     Acc. 60.33 57.20 57.73 58.18 58.20
                  Sparsity     - 70.00 41.07 53.00 78.45
 Llava-OV    VMME     Acc. 58.41 57.70 58.33 58.11 58.70
                  Sparsity     - 70.00 36.80 47.95 81.49
 Llava-OV    MLVU     Acc. 63.32 62.54 62.22 62.50 62.52
                  Sparsity     - 70.00 39.55 35.48 78.34
 Llava-OV     MVB     Acc. 58.38 56.93 56.83 56.75 56.78
                  Sparsity     - 70.00 42.03 63.69 85.49
  MiniCPM    VMME     Acc. 58.81 58.81 58.07 55.89 58.30
                  Sparsity     - 70.00 49.27 57.20 82.87
  MiniCPM    MLVU     Acc. 55.89 54.80 54.84 43.80 53.59
                  Sparsity     - 70.00 41.88